## ETL: Add local timezone to each song played

In [127]:
import os
import lxml

import pandas as pd

#### Use maximum width of notebook

In [128]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [129]:
!whoami

hduser


In [130]:
os.chdir("/data_data/session_length/")
!pwd

/data_data/session_length


In [131]:
!env

LESSOPEN=| /usr/bin/lesspipe %s
MAIL=/var/mail/hduser
USER=hduser
J2SDKDIR=/usr/lib/jvm/java-8-oracle
MPLBACKEND=module://ipykernel.pylab.backend_inline
SHLVL=1
J2REDIR=/usr/lib/jvm/java-8-oracle/jre
OLDPWD=/data_data/session_length/data
HOME=/home/hduser
PAGER=cat
LOGNAME=hduser
_=/usr/bin/nohup
TERM=xterm-color
PATH=/usr/local/bin:/home/hduser/bin:/home/hduser/.local/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/snap/bin:/usr/lib/jvm/java-8-oracle/bin:/usr/lib/jvm/java-8-oracle/db/bin:/usr/lib/jvm/java-8-oracle/jre/bin:/usr/local/hadoop/bin:/usr/local/hadoop/sbin
DERBY_HOME=/usr/lib/jvm/java-8-oracle/db
LANG=en_US.UTF-8
LS_COLORS=rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=

### Make sure no other spark jobs running before doing this

In [132]:
#import pymongo
#import pymongo_spark

#pymongo_spark.activate()


In [133]:
#sc.stop()
#spark.stop()

### If you want to restart Yarn etc

In [134]:
!export HADOOP_HOME=/usr/local/hadoop 
!export PATH=$PATH:/usr/local/hadoop/bin:/usr/local/hadoop/sbin

In [135]:
#!/usr/local/hadoop/sbin/stop-all.sh
#!/usr/local/hadoop/sbin/start-all.sh

In [136]:
APP_NAME = "Session Length Prediction"

try:
    sc.stop()
    spark.stop()
except:
    pass

from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

try:
    sc and spark
except (NameError, UnboundLocalError) as e:
    import findspark
    #findspark.init()
    import pyspark
    import pyspark.sql
    
    #sc = pyspark.SparkContext()
    #spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()
    
sc = SparkContext(conf = SparkConf() .set("spark.sql.autoBroadcastJoinThreshold",-1) .set("spark.driver.maxResultSize", "40g") .set("spark.sql.execution.arrow.enabled", "true") .set('spark.sql.broadcastTimeout', 1000) .set('spark.local.dir', '/data_data/session_length/spark_tmp/') .set('spark.driver.memory', '80G') .set("spark.executor.instances", "20") .set("spark.executor.cores", 20) .set("spark.executor.memory", "12G")).getOrCreate()
spark = SparkSession(sc)
spark.sparkContext.setLogLevel("ERROR")
    

### Check configuration of Spark Environment

In [137]:
sc._conf.getAll()

[(u'spark.local.dir', u'/data_data/session_length/spark_tmp/'),
 (u'spark.sql.execution.arrow.enabled', u'true'),
 (u'spark.driver.port', u'39153'),
 (u'spark.executor.cores', u'20'),
 (u'spark.driver.memory', u'80G'),
 (u'spark.executor.id', u'driver'),
 (u'spark.app.id', u'local-1538061785513'),
 (u'spark.app.name', u'pyspark-shell'),
 (u'spark.executor.instances', u'20'),
 (u'spark.driver.maxResultSize', u'40g'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.driver.host', u'ip-172-31-41-62.ec2.internal'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.sql.autoBroadcastJoinThreshold', u'-1'),
 (u'spark.sql.broadcastTimeout', u'1000'),
 (u'spark.executor.memory', u'12G'),
 (u'spark.ui.showConsoleProgress', u'true')]

In [138]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, DateType, DoubleType, IntegerType
from pyspark.sql.functions import count, mean, stddev_pop, min, max, lit, round, bround, pow, col, corr, lower, upper, avg, stddev, abs, log
from pyspark.sql.functions import lit, trim, rtrim, rpad, trim, coalesce
from pyspark.sql.functions import current_date, current_timestamp, date_add, date_sub, months_between, to_date
from pyspark.sql.functions import udf, col, sum
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, dense_rank, rank, expr

from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression, GBTClassifier, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from datetime import datetime

### Import Data

#### Import utility tools

#### Import both data sets

In [139]:
!head data/lastFM/lastfm-dataset-1K/userid-profile.tsv

#id	gender	age	country	registered
user_000001	m		Japan	Aug 13, 2006
user_000002	f		Peru	Feb 24, 2006
user_000003	m	22	United States	Oct 30, 2005
user_000004	f			Apr 26, 2006
user_000005	m		Bulgaria	Jun 29, 2006
user_000006		24	Russian Federation	May 18, 2006
user_000007	f		United States	Jan 22, 2006
user_000008	m	23	Slovakia	Sep 28, 2006
user_000009	f	19	United States	Jan 13, 2007


In [140]:
users_df = None

In [141]:
from_pattern = 'MMM dd, yyyy'
to_pattern = 'yyyy-MM-dd'

def import_user_data():
    global users_df
    print("==================================================================================================================")
    print("======================================== IMPORTING USERID-PROFILE.CSV  ======================================")
    print("==================================================================================================================\n\n")
    userSchema = StructType([\
        StructField('userid',  StringType(), True),\
        StructField('gender',  StringType(), True),\
        StructField('age',     IntegerType(), True),\
        StructField('country', StringType(), True),\
        StructField('registered', StringType(), True)])
    users = spark.read.format('csv').schema(userSchema).option("sep","\t").load('/data_data/session_length/data/lastFM/lastfm-dataset-1K/userid-profile.tsv')
    users.show(5,False)
    #func = udf(lambda x: datetime.strptime(x, to_pattern), DateType() )
    users_df = users.withColumn('reg_date', to_date(col("registered"), from_pattern))
    users_df.show(5,False)

In [142]:
import_user_data()

======================================== IMPORTING USERID-PROFILE.CSV  ======================================


+-----------+------+----+-------------+------------+
|userid     |gender|age |country      |registered  |
+-----------+------+----+-------------+------------+
|null       |null  |null|null         |null        |
|user_000001|m     |null|Japan        |Aug 13, 2006|
|user_000002|f     |null|Peru         |Feb 24, 2006|
|user_000003|m     |22  |United States|Oct 30, 2005|
|user_000004|f     |null|null         |Apr 26, 2006|
+-----------+------+----+-------------+------------+
only showing top 5 rows

+-----------+------+----+-------------+------------+----------+
|userid     |gender|age |country      |registered  |reg_date  |
+-----------+------+----+-------------+------------+----------+
|null       |null  |null|null         |null        |null      |
|user_000001|m     |null|Japan        |Aug 13, 2006|2006-08-13|
|user_000002|f     |null|Peru         |Feb 24, 2006|2006-02-24|
|u

#### Import music_data

In [143]:
!head data/lastFM/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv

user_000001	2009-05-04T23:08:57Z	f1b1cf71-bd35-4e99-8624-24a6e15f133a	Deep Dish		Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
user_000001	2009-05-04T13:54:10Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Composition 0919 (Live_2009_4_15)
user_000001	2009-05-04T13:52:04Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc2 (Live_2009_4_15)
user_000001	2009-05-04T13:42:52Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Hibari (Live_2009_4_15)
user_000001	2009-05-04T13:42:11Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc1 (Live_2009_4_15)
user_000001	2009-05-04T13:38:31Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		To Stanford (Live_2009_4_15)
user_000001	2009-05-04T13:33:28Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Improvisation (Live_2009_4_15)
user_000001	2009-05-04T13:23:45Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Glacier (Live_2009_4_15)
user_000001	2009-05-04T13:19:22Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Parolibre (Live_2009_4_15)
user_000001	2009-05-04T13:13:38Z	a7f7df4a-77d8-4f12-8acd

In [144]:
sessions_DF = None
sessions_df = None

In [145]:
from_pattern = 'yyyy-MM-ddTHH:mm:ssZ'
to_pattern = 'yyyy-MM-dd'

def import_session_data():
    global sessions_DF
    global sessions_df
    print("====================================================================================================================")
    print("====================================userid-timestamp-artid-artname-traid-traname.tsv================================")
    print("==================================================================================================================\n\n")
    userSchema = StructType([\
        StructField('userid',  StringType(), True),\
        StructField('timestamp',  StringType(), True),\
        StructField('artid',     StringType(), True),\
        StructField('artname_', StringType(), True),\
        StructField('traid', StringType(), True),\
        StructField('traname_', StringType(), True)
        ])
    sessions = spark.read.format('csv').schema(userSchema).option("sep","\t").load('/data_data/session_length/data/lastFM/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv')
    sessions.show(5,False)
    #Assign window to time (using hour as window....)
    def get_window(x):
        return int(str(x).split("T")[1].split(":")[0])
    getWindow = udf(lambda timestamp:  get_window(timestamp))
    sessions_DF = sessions.withColumn('session_date',to_date(col('timestamp'))).withColumn('session_window', getWindow(col('timestamp')))
    sessions_DF.show(5,False)
    #Translate artname_ to English
    def translate_to_english(str_to_translate):
        return gs.translate(str_to_translate, 'en')
    func = udf(lambda str_to_translate:  translate_to_english(str_to_translate), StringType())
    #sessions_df = sessions_DF.withColumn('artname', translate_to_english('artname_'))
    #sessions_df[["artname_"]].apply(lambda row: np.mean(row),axis=0)
    #sessions_df["artname"] = sessions_DF["artname_"].apply(lambda name: translate_to_english(name),axis=0)
    #.withColumn('traname', func(col('traname_')))
    sessions_df = sessions_DF.rdd.map(lambda x: (x["artname_"]))
    print(sessions_df.take(5))

In [146]:
import_session_data()

====================================userid-timestamp-artid-artname-traid-traname.tsv================================


+-----------+--------------------+------------------------------------+---------+-----+------------------------------------------+
|userid     |timestamp           |artid                               |artname_ |traid|traname_                                  |
+-----------+--------------------+------------------------------------+---------+-----+------------------------------------------+
|user_000001|2009-05-04T23:08:57Z|f1b1cf71-bd35-4e99-8624-24a6e15f133a|Deep Dish|null |Fuck Me Im Famous (Pacha Ibiza)-09-28-2007|
|user_000001|2009-05-04T13:54:10Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一     |null |Composition 0919 (Live_2009_4_15)         |
|user_000001|2009-05-04T13:52:04Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一     |null |Mc2 (Live_2009_4_15)                      |
|user_000001|2009-05-04T13:42:52Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一     |null |Hibari (

In [147]:
#from pyspark.sql.types import Row

#row = Row("artname_")
#df_sessions = sessions_df.map(row).toDF()
#schema = StructType([StructField("artname", StringType(), True)])
#df_sessions = sessions_df.toDF(["artname"],)
#[gs.translate(x,'en') for x in df_sessions]
#df_sessions = sessions_df.flatMap(gs.translate)
#df_sessions.take(5)

In [148]:
users_df.show()

+-----------+------+----+------------------+------------+----------+
|     userid|gender| age|           country|  registered|  reg_date|
+-----------+------+----+------------------+------------+----------+
|       null|  null|null|              null|        null|      null|
|user_000001|     m|null|             Japan|Aug 13, 2006|2006-08-13|
|user_000002|     f|null|              Peru|Feb 24, 2006|2006-02-24|
|user_000003|     m|  22|     United States|Oct 30, 2005|2005-10-30|
|user_000004|     f|null|              null|Apr 26, 2006|2006-04-26|
|user_000005|     m|null|          Bulgaria|Jun 29, 2006|2006-06-29|
|user_000006|  null|  24|Russian Federation|May 18, 2006|2006-05-18|
|user_000007|     f|null|     United States|Jan 22, 2006|2006-01-22|
|user_000008|     m|  23|          Slovakia|Sep 28, 2006|2006-09-28|
|user_000009|     f|  19|     United States|Jan 13, 2007|2007-01-13|
|user_000010|     m|  19|            Poland| May 4, 2006|2006-05-04|
|user_000011|     m|  21|         

#### Denormalize by joining user and session databases

In [149]:
users_df = users_df.withColumnRenamed('userid','user_id')
u = users_df.alias('u')
s = sessions_DF.alias('s')
join_condition = [ (u.user_id == s.userid) ]
sessionsDF = s.join(u, join_condition, 'inner').drop('user_id')
sessionsDF.show(20,False)

+-----------+--------------------+------------------------------------+------------------+------------------------------------+---------------------------+------------+--------------+------+---+-------------+-----------+----------+
|userid     |timestamp           |artid                               |artname_          |traid                               |traname_                   |session_date|session_window|gender|age|country      |registered |reg_date  |
+-----------+--------------------+------------------------------------+------------------+------------------------------------+---------------------------+------------+--------------+------+---+-------------+-----------+----------+
|user_000066|2009-03-27T17:18:26Z|null                                |The Amen Corner   |null                                |Our Love (Is In The Pocket)|2009-03-27  |17            |f     |20 |United States|May 9, 2006|2006-05-09|
|user_000066|2009-03-08T04:06:21Z|2ceb4e66-4eaa-4dba-ad3a-30df3b742557|N

#### Safe as TSV file for Time Series analysis

#### Uncomment if you want a fresh copy of the file

In [150]:
#!rm -R /data_data/session_length/data/tmp
#sessionsDF.coalesce(1).write.option("delimiter", "\t").csv('/data_data/session_length/data/tmp', header="True")
#!mv /data_data/session_length/data/tmp/part-* /data_data/session_length/data/Session_data.csv

In [151]:
!head /data_data/session_length/data/Session_data.csv

userid	timestamp	artid	artname_	traid	traname_	session_window	gender	age	country	registered	reg_date
user_000001	2009-05-04T23:08:57Z	f1b1cf71-bd35-4e99-8624-24a6e15f133a	Deep Dish		Fuck Me Im Famous (Pacha Ibiza)-09-28-2007	23	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:54:10Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Composition 0919 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:52:04Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc2 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:42:52Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Hibari (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:42:11Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc1 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:38:31Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		To Stanford (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:33:28Z	a7f7df4a-77d8-4f1

### Get Local time by converting from UTC to Local [WORK IN PROGRESS]

In [152]:
from dateutil import tz
import pytz

### Use countryInfo from here: https://gist.githubusercontent.com/pamelafox/986163/raw/f5f9db4f1b287804fd07ffb3296ed0036292bc7a/countryinfo.py

In [153]:
countries_info = [
{'timezones': ['Europe/Andorra'], 'code': 'AD', 'continent': 'Europe', 'name': 'Andorra', 'capital': 'Andorra la Vella'},
{'timezones': ['Asia/Kabul'], 'code': 'AF', 'continent': 'Asia', 'name': 'Afghanistan', 'capital': 'Kabul'},
{'timezones': ['America/Antigua'], 'code': 'AG', 'continent': 'North America', 'name': 'Antigua and Barbuda', 'capital': "St. John's"},
{'timezones': ['Europe/Tirane'], 'code': 'AL', 'continent': 'Europe', 'name': 'Albania', 'capital': 'Tirana'},
{'timezones': ['Asia/Yerevan'], 'code': 'AM', 'continent': 'Asia', 'name': 'Armenia', 'capital': 'Yerevan'},
{'timezones': ['Africa/Luanda'], 'code': 'AO', 'continent': 'Africa', 'name': 'Angola', 'capital': 'Luanda'},
{'timezones': ['America/Argentina/Buenos_Aires', 'America/Argentina/Cordoba', 'America/Argentina/Jujuy', 'America/Argentina/Tucuman', 'America/Argentina/Catamarca', 'America/Argentina/La_Rioja', 'America/Argentina/San_Juan', 'America/Argentina/Mendoza', 'America/Argentina/Rio_Gallegos', 'America/Argentina/Ushuaia'], 'code': 'AR', 'continent': 'South America', 'name': 'Argentina', 'capital': 'Buenos Aires'},
{'timezones': ['Europe/Vienna'], 'code': 'AT', 'continent': 'Europe', 'name': 'Austria', 'capital': 'Vienna'},
{'timezones': ['Australia/Lord_Howe', 'Australia/Hobart', 'Australia/Currie', 'Australia/Melbourne', 'Australia/Sydney', 'Australia/Broken_Hill', 'Australia/Brisbane', 'Australia/Lindeman', 'Australia/Adelaide', 'Australia/Darwin', 'Australia/Perth'], 'code': 'AU', 'continent': 'Oceania', 'name': 'Australia', 'capital': 'Canberra'},
{'timezones': ['Asia/Baku'], 'code': 'AZ', 'continent': 'Asia', 'name': 'Azerbaijan', 'capital': 'Baku'},
{'timezones': ['America/Barbados'], 'code': 'BB', 'continent': 'North America', 'name': 'Barbados', 'capital': 'Bridgetown'},
{'timezones': ['Asia/Dhaka'], 'code': 'BD', 'continent': 'Asia', 'name': 'Bangladesh', 'capital': 'Dhaka'},
{'timezones': ['Europe/Brussels'], 'code': 'BE', 'continent': 'Europe', 'name': 'Belgium', 'capital': 'Brussels'},
{'timezones': ['Africa/Ouagadougou'], 'code': 'BF', 'continent': 'Africa', 'name': 'Burkina Faso', 'capital': 'Ouagadougou'},
{'timezones': ['Europe/Sofia'], 'code': 'BG', 'continent': 'Europe', 'name': 'Bulgaria', 'capital': 'Sofia'},
{'timezones': ['Asia/Bahrain'], 'code': 'BH', 'continent': 'Asia', 'name': 'Bahrain', 'capital': 'Manama'},
{'timezones': ['Africa/Bujumbura'], 'code': 'BI', 'continent': 'Africa', 'name': 'Burundi', 'capital': 'Bujumbura'},
{'timezones': ['Africa/Porto-Novo'], 'code': 'BJ', 'continent': 'Africa', 'name': 'Benin', 'capital': 'Porto-Novo'},
{'timezones': ['Asia/Brunei'], 'code': 'BN', 'continent': 'Asia', 'name': 'Brunei Darussalam', 'capital': 'Bandar Seri Begawan'},
{'timezones': ['America/La_Paz'], 'code': 'BO', 'continent': 'South America', 'name': 'Bolivia', 'capital': 'Sucre'},
{'timezones': ['America/Noronha', 'America/Belem', 'America/Fortaleza', 'America/Recife', 'America/Araguaina', 'America/Maceio', 'America/Bahia', 'America/Sao_Paulo', 'America/Campo_Grande', 'America/Cuiaba', 'America/Porto_Velho', 'America/Boa_Vista', 'America/Manaus', 'America/Eirunepe', 'America/Rio_Branco'], 'code': 'BR', 'continent': 'South America', 'name': 'Brazil', 'capital': 'Bras\xc3\xadlia'},
{'timezones': ['America/Nassau'], 'code': 'BS', 'continent': 'North America', 'name': 'Bahamas', 'capital': 'Nassau'},
{'timezones': ['Asia/Thimphu'], 'code': 'BT', 'continent': 'Asia', 'name': 'Bhutan', 'capital': 'Thimphu'},
{'timezones': ['Africa/Gaborone'], 'code': 'BW', 'continent': 'Africa', 'name': 'Botswana', 'capital': 'Gaborone'},
{'timezones': ['Europe/Minsk'], 'code': 'BY', 'continent': 'Europe', 'name': 'Belarus', 'capital': 'Minsk'},
{'timezones': ['America/Belize'], 'code': 'BZ', 'continent': 'North America', 'name': 'Belize', 'capital': 'Belmopan'},
{'timezones': ['America/St_Johns', 'America/Halifax', 'America/Glace_Bay', 'America/Moncton', 'America/Goose_Bay', 'America/Blanc-Sablon', 'America/Montreal', 'America/Toronto', 'America/Nipigon', 'America/Thunder_Bay', 'America/Pangnirtung', 'America/Iqaluit', 'America/Atikokan', 'America/Rankin_Inlet', 'America/Winnipeg', 'America/Rainy_River', 'America/Cambridge_Bay', 'America/Regina', 'America/Swift_Current', 'America/Edmonton', 'America/Yellowknife', 'America/Inuvik', 'America/Dawson_Creek', 'America/Vancouver', 'America/Whitehorse', 'America/Dawson'], 'code': 'CA', 'continent': 'North America', 'name': 'Canada', 'capital': 'Ottawa'},
{'timezones': ['Africa/Kinshasa', 'Africa/Lubumbashi'], 'code': 'CD', 'continent': 'Africa', 'name': 'Democratic Republic of the Congo', 'capital': 'Kinshasa'},
{'timezones': ['Africa/Brazzaville'], 'code': 'CG', 'continent': 'Africa', 'name': 'Republic of the Congo', 'capital': 'Brazzaville'},
{'timezones': ['Africa/Abidjan'], 'code': 'CI', 'continent': 'Africa', 'name': "C\xc3\xb4te d'Ivoire", 'capital': 'Yamoussoukro'},
{'timezones': ['America/Santiago', 'Pacific/Easter'], 'code': 'CL', 'continent': 'South America', 'name': 'Chile', 'capital': 'Santiago'},
{'timezones': ['Africa/Douala'], 'code': 'CM', 'continent': 'Africa', 'name': 'Cameroon', 'capital': 'Yaound\xc3\xa9'},
{'timezones': ['Asia/Shanghai', 'Asia/Harbin', 'Asia/Chongqing', 'Asia/Urumqi', 'Asia/Kashgar'], 'code': 'CN', 'continent': 'Asia', 'name': "People's Republic of China", 'capital': 'Beijing'},
{'timezones': ['America/Bogota'], 'code': 'CO', 'continent': 'South America', 'name': 'Colombia', 'capital': 'Bogot\xc3\xa1'},
{'timezones': ['America/Costa_Rica'], 'code': 'CR', 'continent': 'North America', 'name': 'Costa Rica', 'capital': 'San Jos\xc3\xa9'},
{'timezones': ['America/Havana'], 'code': 'CU', 'continent': 'North America', 'name': 'Cuba', 'capital': 'Havana'},
{'timezones': ['Atlantic/Cape_Verde'], 'code': 'CV', 'continent': 'Africa', 'name': 'Cape Verde', 'capital': 'Praia'},
{'timezones': ['Asia/Nicosia'], 'code': 'CY', 'continent': 'Asia', 'name': 'Cyprus', 'capital': 'Nicosia'},
{'timezones': ['Europe/Prague'], 'code': 'CZ', 'continent': 'Europe', 'name': 'Czech Republic', 'capital': 'Prague'},
{'timezones': ['Europe/Berlin'], 'code': 'DE', 'continent': 'Europe', 'name': 'Germany', 'capital': 'Berlin'},
{'timezones': ['Africa/Djibouti'], 'code': 'DJ', 'continent': 'Africa', 'name': 'Djibouti', 'capital': 'Djibouti City'},
{'timezones': ['Europe/Copenhagen'], 'code': 'DK', 'continent': 'Europe', 'name': 'Denmark', 'capital': 'Copenhagen'},
{'timezones': ['America/Dominica'], 'code': 'DM', 'continent': 'North America', 'name': 'Dominica', 'capital': 'Roseau'},
{'timezones': ['America/Santo_Domingo'], 'code': 'DO', 'continent': 'North America', 'name': 'Dominican Republic', 'capital': 'Santo Domingo'},
{'timezones': ['America/Guayaquil', 'Pacific/Galapagos'], 'code': 'EC', 'continent': 'South America', 'name': 'Ecuador', 'capital': 'Quito'},
{'timezones': ['Europe/Tallinn'], 'code': 'EE', 'continent': 'Europe', 'name': 'Estonia', 'capital': 'Tallinn'},
{'timezones': ['Africa/Cairo'], 'code': 'EG', 'continent': 'Africa', 'name': 'Egypt', 'capital': 'Cairo'},
{'timezones': ['Africa/Asmera'], 'code': 'ER', 'continent': 'Africa', 'name': 'Eritrea', 'capital': 'Asmara'},
{'timezones': ['Africa/Addis_Ababa'], 'code': 'ET', 'continent': 'Africa', 'name': 'Ethiopia', 'capital': 'Addis Ababa'},
{'timezones': ['Europe/Helsinki'], 'code': 'FI', 'continent': 'Europe', 'name': 'Finland', 'capital': 'Helsinki'},
{'timezones': ['Pacific/Fiji'], 'code': 'FJ', 'continent': 'Oceania', 'name': 'Fiji', 'capital': 'Suva'},
{'timezones': ['Europe/Paris'], 'code': 'FR', 'continent': 'Europe', 'name': 'France', 'capital': 'Paris'},
{'timezones': ['Africa/Libreville'], 'code': 'GA', 'continent': 'Africa', 'name': 'Gabon', 'capital': 'Libreville'},
{'timezones': ['Asia/Tbilisi'], 'code': 'GE', 'continent': 'Asia', 'name': 'Georgia', 'capital': 'Tbilisi'},
{'timezones': ['Africa/Accra'], 'code': 'GH', 'continent': 'Africa', 'name': 'Ghana', 'capital': 'Accra'},
{'timezones': ['Africa/Banjul'], 'code': 'GM', 'continent': 'Africa', 'name': 'The Gambia', 'capital': 'Banjul'},
{'timezones': ['Africa/Conakry'], 'code': 'GN', 'continent': 'Africa', 'name': 'Guinea', 'capital': 'Conakry'},
{'timezones': ['Europe/Athens'], 'code': 'GR', 'continent': 'Europe', 'name': 'Greece', 'capital': 'Athens'},
{'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'North America', 'name': 'Guatemala', 'capital': 'Guatemala City'},
{'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'North America', 'name': 'Haiti', 'capital': 'Port-au-Prince'},
{'timezones': ['Africa/Bissau'], 'code': 'GW', 'continent': 'Africa', 'name': 'Guinea-Bissau', 'capital': 'Bissau'},
{'timezones': ['America/Guyana'], 'code': 'GY', 'continent': 'South America', 'name': 'Guyana', 'capital': 'Georgetown'},
{'timezones': ['America/Tegucigalpa'], 'code': 'HN', 'continent': 'North America', 'name': 'Honduras', 'capital': 'Tegucigalpa'},
{'timezones': ['Europe/Budapest'], 'code': 'HU', 'continent': 'Europe', 'name': 'Hungary', 'capital': 'Budapest'},
{'timezones': ['Asia/Jakarta', 'Asia/Pontianak', 'Asia/Makassar', 'Asia/Jayapura'], 'code': 'ID', 'continent': 'Asia', 'name': 'Indonesia', 'capital': 'Jakarta'},
{'timezones': ['Europe/Dublin'], 'code': 'IE', 'continent': 'Europe', 'name': 'Republic of Ireland', 'capital': 'Dublin'},
{'timezones': ['Asia/Jerusalem'], 'code': 'IL', 'continent': 'Asia', 'name': 'Israel', 'capital': 'Jerusalem'},
{'timezones': ['Asia/Calcutta'], 'code': 'IN', 'continent': 'Asia', 'name': 'India', 'capital': 'New Delhi'},
{'timezones': ['Asia/Baghdad'], 'code': 'IQ', 'continent': 'Asia', 'name': 'Iraq', 'capital': 'Baghdad'},
{'timezones': ['Asia/Tehran'], 'code': 'IR', 'continent': 'Asia', 'name': 'Iran', 'capital': 'Tehran'},
{'timezones': ['Atlantic/Reykjavik'], 'code': 'IS', 'continent': 'Europe', 'name': 'Iceland', 'capital': 'Reykjav\xc3\xadk'},
{'timezones': ['Europe/Rome'], 'code': 'IT', 'continent': 'Europe', 'name': 'Italy', 'capital': 'Rome'},
{'timezones': ['America/Jamaica'], 'code': 'JM', 'continent': 'North America', 'name': 'Jamaica', 'capital': 'Kingston'},
{'timezones': ['Asia/Amman'], 'code': 'JO', 'continent': 'Asia', 'name': 'Jordan', 'capital': 'Amman'},
{'timezones': ['Asia/Tokyo'], 'code': 'JP', 'continent': 'Asia', 'name': 'Japan', 'capital': 'Tokyo'},
{'timezones': ['Africa/Nairobi'], 'code': 'KE', 'continent': 'Africa', 'name': 'Kenya', 'capital': 'Nairobi'},
{'timezones': ['Asia/Bishkek'], 'code': 'KG', 'continent': 'Asia', 'name': 'Kyrgyzstan', 'capital': 'Bishkek'},
{'timezones': ['Pacific/Tarawa', 'Pacific/Enderbury', 'Pacific/Kiritimati'], 'code': 'KI', 'continent': 'Oceania', 'name': 'Kiribati', 'capital': 'Tarawa'},
{'timezones': ['Asia/Pyongyang'], 'code': 'KP', 'continent': 'Asia', 'name': 'North Korea', 'capital': 'Pyongyang'},
{'timezones': ['Asia/Seoul'], 'code': 'KR', 'continent': 'Asia', 'name': 'South Korea', 'capital': 'Seoul'},
{'timezones': ['Asia/Kuwait'], 'code': 'KW', 'continent': 'Asia', 'name': 'Kuwait', 'capital': 'Kuwait City'},
{'timezones': ['Asia/Beirut'], 'code': 'LB', 'continent': 'Asia', 'name': 'Lebanon', 'capital': 'Beirut'},
{'timezones': ['Europe/Vaduz'], 'code': 'LI', 'continent': 'Europe', 'name': 'Liechtenstein', 'capital': 'Vaduz'},
{'timezones': ['Africa/Monrovia'], 'code': 'LR', 'continent': 'Africa', 'name': 'Liberia', 'capital': 'Monrovia'},
{'timezones': ['Africa/Maseru'], 'code': 'LS', 'continent': 'Africa', 'name': 'Lesotho', 'capital': 'Maseru'},
{'timezones': ['Europe/Vilnius'], 'code': 'LT', 'continent': 'Europe', 'name': 'Lithuania', 'capital': 'Vilnius'},
{'timezones': ['Europe/Luxembourg'], 'code': 'LU', 'continent': 'Europe', 'name': 'Luxembourg', 'capital': 'Luxembourg City'},
{'timezones': ['Europe/Riga'], 'code': 'LV', 'continent': 'Europe', 'name': 'Latvia', 'capital': 'Riga'},
{'timezones': ['Africa/Tripoli'], 'code': 'LY', 'continent': 'Africa', 'name': 'Libya', 'capital': 'Tripoli'},
{'timezones': ['Indian/Antananarivo'], 'code': 'MG', 'continent': 'Africa', 'name': 'Madagascar', 'capital': 'Antananarivo'},
{'timezones': ['Pacific/Majuro', 'Pacific/Kwajalein'], 'code': 'MH', 'continent': 'Oceania', 'name': 'Marshall Islands', 'capital': 'Majuro'},
{'timezones': ['Europe/Skopje'], 'code': 'MK', 'continent': 'Europe', 'name': 'Macedonia', 'capital': 'Skopje'},
{'timezones': ['Africa/Bamako'], 'code': 'ML', 'continent': 'Africa', 'name': 'Mali', 'capital': 'Bamako'},
{'timezones': ['Asia/Rangoon'], 'code': 'MM', 'continent': 'Asia', 'name': 'Myanmar', 'capital': 'Naypyidaw'},
{'timezones': ['Asia/Ulaanbaatar', 'Asia/Hovd', 'Asia/Choibalsan'], 'code': 'MN', 'continent': 'Asia', 'name': 'Mongolia', 'capital': 'Ulaanbaatar'},
{'timezones': ['Africa/Nouakchott'], 'code': 'MR', 'continent': 'Africa', 'name': 'Mauritania', 'capital': 'Nouakchott'},
{'timezones': ['Europe/Malta'], 'code': 'MT', 'continent': 'Europe', 'name': 'Malta', 'capital': 'Valletta'},
{'timezones': ['Indian/Mauritius'], 'code': 'MU', 'continent': 'Africa', 'name': 'Mauritius', 'capital': 'Port Louis'},
{'timezones': ['Indian/Maldives'], 'code': 'MV', 'continent': 'Asia', 'name': 'Maldives', 'capital': 'Mal\xc3\xa9'},
{'timezones': ['Africa/Blantyre'], 'code': 'MW', 'continent': 'Africa', 'name': 'Malawi', 'capital': 'Lilongwe'},
{'timezones': ['America/Mexico_City', 'America/Cancun', 'America/Merida', 'America/Monterrey', 'America/Mazatlan', 'America/Chihuahua', 'America/Hermosillo', 'America/Tijuana'], 'code': 'MX', 'continent': 'North America', 'name': 'Mexico', 'capital': 'Mexico City'},
{'timezones': ['Asia/Kuala_Lumpur', 'Asia/Kuching'], 'code': 'MY', 'continent': 'Asia', 'name': 'Malaysia', 'capital': 'Kuala Lumpur'},
{'timezones': ['Africa/Maputo'], 'code': 'MZ', 'continent': 'Africa', 'name': 'Mozambique', 'capital': 'Maputo'},
{'timezones': ['Africa/Windhoek'], 'code': 'NA', 'continent': 'Africa', 'name': 'Namibia', 'capital': 'Windhoek'},
{'timezones': ['Africa/Niamey'], 'code': 'NE', 'continent': 'Africa', 'name': 'Niger', 'capital': 'Niamey'},
{'timezones': ['Africa/Lagos'], 'code': 'NG', 'continent': 'Africa', 'name': 'Nigeria', 'capital': 'Abuja'},
{'timezones': ['America/Managua'], 'code': 'NI', 'continent': 'North America', 'name': 'Nicaragua', 'capital': 'Managua'},
{'timezones': ['Europe/Amsterdam'], 'code': 'NL', 'continent': 'Europe', 'name': 'Kingdom of the Netherlands', 'capital': 'Amsterdam'},
{'timezones': ['Europe/Oslo'], 'code': 'NO', 'continent': 'Europe', 'name': 'Norway', 'capital': 'Oslo'},
{'timezones': ['Asia/Katmandu'], 'code': 'NP', 'continent': 'Asia', 'name': 'Nepal', 'capital': 'Kathmandu'},
{'timezones': ['Pacific/Nauru'], 'code': 'NR', 'continent': 'Oceania', 'name': 'Nauru', 'capital': 'Yaren'},
{'timezones': ['Pacific/Auckland', 'Pacific/Chatham'], 'code': 'NZ', 'continent': 'Oceania', 'name': 'New Zealand', 'capital': 'Wellington'},
{'timezones': ['Asia/Muscat'], 'code': 'OM', 'continent': 'Asia', 'name': 'Oman', 'capital': 'Muscat'},
{'timezones': ['America/Panama'], 'code': 'PA', 'continent': 'North America', 'name': 'Panama', 'capital': 'Panama City'},
{'timezones': ['America/Lima'], 'code': 'PE', 'continent': 'South America', 'name': 'Peru', 'capital': 'Lima'},
{'timezones': ['Pacific/Port_Moresby'], 'code': 'PG', 'continent': 'Oceania', 'name': 'Papua New Guinea', 'capital': 'Port Moresby'},
{'timezones': ['Asia/Manila'], 'code': 'PH', 'continent': 'Asia', 'name': 'Philippines', 'capital': 'Manila'},
{'timezones': ['Asia/Karachi'], 'code': 'PK', 'continent': 'Asia', 'name': 'Pakistan', 'capital': 'Islamabad'},
{'timezones': ['Europe/Warsaw'], 'code': 'PL', 'continent': 'Europe', 'name': 'Poland', 'capital': 'Warsaw'},
{'timezones': ['Europe/Lisbon', 'Atlantic/Madeira', 'Atlantic/Azores'], 'code': 'PT', 'continent': 'Europe', 'name': 'Portugal', 'capital': 'Lisbon'},
{'timezones': ['Pacific/Palau'], 'code': 'PW', 'continent': 'Oceania', 'name': 'Palau', 'capital': 'Ngerulmud'},
{'timezones': ['America/Asuncion'], 'code': 'PY', 'continent': 'South America', 'name': 'Paraguay', 'capital': 'Asunci\xc3\xb3n'},
{'timezones': ['Asia/Qatar'], 'code': 'QA', 'continent': 'Asia', 'name': 'Qatar', 'capital': 'Doha'},
{'timezones': ['Europe/Bucharest'], 'code': 'RO', 'continent': 'Europe', 'name': 'Romania', 'capital': 'Bucharest'},
{'timezones': ['Europe/Kaliningrad', 'Europe/Moscow', 'Europe/Volgograd', 'Europe/Samara', 'Asia/Yekaterinburg', 'Asia/Omsk', 'Asia/Novosibirsk', 'Asia/Krasnoyarsk', 'Asia/Irkutsk', 'Asia/Yakutsk', 'Asia/Vladivostok', 'Asia/Sakhalin', 'Asia/Magadan', 'Asia/Kamchatka', 'Asia/Anadyr'], 'code': 'RU', 'continent': 'Europe', 'name': 'Russia', 'capital': 'Moscow'},
{'timezones': ['Africa/Kigali'], 'code': 'RW', 'continent': 'Africa', 'name': 'Rwanda', 'capital': 'Kigali'},
{'timezones': ['Asia/Riyadh'], 'code': 'SA', 'continent': 'Asia', 'name': 'Saudi Arabia', 'capital': 'Riyadh'},
{'timezones': ['Pacific/Guadalcanal'], 'code': 'SB', 'continent': 'Oceania', 'name': 'Solomon Islands', 'capital': 'Honiara'},
{'timezones': ['Indian/Mahe'], 'code': 'SC', 'continent': 'Africa', 'name': 'Seychelles', 'capital': 'Victoria'},
{'timezones': ['Africa/Khartoum'], 'code': 'SD', 'continent': 'Africa', 'name': 'Sudan', 'capital': 'Khartoum'},
{'timezones': ['Europe/Stockholm'], 'code': 'SE', 'continent': 'Europe', 'name': 'Sweden', 'capital': 'Stockholm'},
{'timezones': ['Asia/Singapore'], 'code': 'SG', 'continent': 'Asia', 'name': 'Singapore', 'capital': 'Singapore'},
{'timezones': ['Europe/Ljubljana'], 'code': 'SI', 'continent': 'Europe', 'name': 'Slovenia', 'capital': 'Ljubljana'},
{'timezones': ['Europe/Bratislava'], 'code': 'SK', 'continent': 'Europe', 'name': 'Slovakia', 'capital': 'Bratislava'},
{'timezones': ['Africa/Freetown'], 'code': 'SL', 'continent': 'Africa', 'name': 'Sierra Leone', 'capital': 'Freetown'},
{'timezones': ['Europe/San_Marino'], 'code': 'SM', 'continent': 'Europe', 'name': 'San Marino', 'capital': 'San Marino'},
{'timezones': ['Africa/Dakar'], 'code': 'SN', 'continent': 'Africa', 'name': 'Senegal', 'capital': 'Dakar'},
{'timezones': ['Africa/Mogadishu'], 'code': 'SO', 'continent': 'Africa', 'name': 'Somalia', 'capital': 'Mogadishu'},
{'timezones': ['America/Paramaribo'], 'code': 'SR', 'continent': 'South America', 'name': 'Suriname', 'capital': 'Paramaribo'},
{'timezones': ['Africa/Sao_Tome'], 'code': 'ST', 'continent': 'Africa', 'name': 'S\xc3\xa3o Tom\xc3\xa9 and Pr\xc3\xadncipe', 'capital': 'S\xc3\xa3o Tom\xc3\xa9'},
{'timezones': ['Asia/Damascus'], 'code': 'SY', 'continent': 'Asia', 'name': 'Syria', 'capital': 'Damascus'},
{'timezones': ['Africa/Lome'], 'code': 'TG', 'continent': 'Africa', 'name': 'Togo', 'capital': 'Lom\xc3\xa9'},
{'timezones': ['Asia/Bangkok'], 'code': 'TH', 'continent': 'Asia', 'name': 'Thailand', 'capital': 'Bangkok'},
{'timezones': ['Asia/Dushanbe'], 'code': 'TJ', 'continent': 'Asia', 'name': 'Tajikistan', 'capital': 'Dushanbe'},
{'timezones': ['Asia/Ashgabat'], 'code': 'TM', 'continent': 'Asia', 'name': 'Turkmenistan', 'capital': 'Ashgabat'},
{'timezones': ['Africa/Tunis'], 'code': 'TN', 'continent': 'Africa', 'name': 'Tunisia', 'capital': 'Tunis'},
{'timezones': ['Pacific/Tongatapu'], 'code': 'TO', 'continent': 'Oceania', 'name': 'Tonga', 'capital': 'Nuku\xca\xbbalofa'},
{'timezones': ['Europe/Istanbul'], 'code': 'TR', 'continent': 'Asia', 'name': 'Turkey', 'capital': 'Ankara'},
{'timezones': ['America/Port_of_Spain'], 'code': 'TT', 'continent': 'North America', 'name': 'Trinidad and Tobago', 'capital': 'Port of Spain'},
{'timezones': ['Pacific/Funafuti'], 'code': 'TV', 'continent': 'Oceania', 'name': 'Tuvalu', 'capital': 'Funafuti'},
{'timezones': ['Africa/Dar_es_Salaam'], 'code': 'TZ', 'continent': 'Africa', 'name': 'Tanzania', 'capital': 'Dodoma'},
{'timezones': ['Europe/Kiev', 'Europe/Uzhgorod', 'Europe/Zaporozhye', 'Europe/Simferopol'], 'code': 'UA', 'continent': 'Europe', 'name': 'Ukraine', 'capital': 'Kiev'},
{'timezones': ['Africa/Kampala'], 'code': 'UG', 'continent': 'Africa', 'name': 'Uganda', 'capital': 'Kampala'},
{'timezones': ['America/New_York', 'America/Detroit', 'America/Kentucky/Louisville', 'America/Kentucky/Monticello', 'America/Indiana/Indianapolis', 'America/Indiana/Marengo', 'America/Indiana/Knox', 'America/Indiana/Vevay', 'America/Chicago', 'America/Indiana/Vincennes', 'America/Indiana/Petersburg', 'America/Menominee', 'America/North_Dakota/Center', 'America/North_Dakota/New_Salem', 'America/Denver', 'America/Boise', 'America/Shiprock', 'America/Phoenix', 'America/Los_Angeles', 'America/Anchorage', 'America/Juneau', 'America/Yakutat', 'America/Nome', 'America/Adak', 'Pacific/Honolulu'], 'code': 'US', 'continent': 'North America', 'name': 'United States', 'capital': 'Washington, D.C.'},
{'timezones': ['America/Montevideo'], 'code': 'UY', 'continent': 'South America', 'name': 'Uruguay', 'capital': 'Montevideo'},
{'timezones': ['Asia/Samarkand', 'Asia/Tashkent'], 'code': 'UZ', 'continent': 'Asia', 'name': 'Uzbekistan', 'capital': 'Tashkent'},
{'timezones': ['Europe/Vatican'], 'code': 'VA', 'continent': 'Europe', 'name': 'Vatican City', 'capital': 'Vatican City'},
{'timezones': ['America/Caracas'], 'code': 'VE', 'continent': 'South America', 'name': 'Venezuela', 'capital': 'Caracas'},
{'timezones': ['Asia/Saigon'], 'code': 'VN', 'continent': 'Asia', 'name': 'Vietnam', 'capital': 'Hanoi'},
{'timezones': ['Pacific/Efate'], 'code': 'VU', 'continent': 'Oceania', 'name': 'Vanuatu', 'capital': 'Port Vila'},
{'timezones': ['Asia/Aden'], 'code': 'YE', 'continent': 'Asia', 'name': 'Yemen', 'capital': "Sana'a"},
{'timezones': ['Africa/Lusaka'], 'code': 'ZM', 'continent': 'Africa', 'name': 'Zambia', 'capital': 'Lusaka'},
{'timezones': ['Africa/Harare'], 'code': 'ZW', 'continent': 'Africa', 'name': 'Zimbabwe', 'capital': 'Harare'},
{'timezones': ['Africa/Algiers'], 'code': 'DZ', 'continent': 'Africa', 'name': 'Algeria', 'capital': 'Algiers'},
{'timezones': ['Europe/Sarajevo'], 'code': 'BA', 'continent': 'Europe', 'name': 'Bosnia and Herzegovina', 'capital': 'Sarajevo'},
{'timezones': ['Asia/Phnom_Penh'], 'code': 'KH', 'continent': 'Asia', 'name': 'Cambodia', 'capital': 'Phnom Penh'},
{'timezones': ['Africa/Bangui'], 'code': 'CF', 'continent': 'Africa', 'name': 'Central African Republic', 'capital': 'Bangui'},
{'timezones': ['Africa/Ndjamena'], 'code': 'TD', 'continent': 'Africa', 'name': 'Chad', 'capital': "N'Djamena"},
{'timezones': ['Indian/Comoro'], 'code': 'KM', 'continent': 'Africa', 'name': 'Comoros', 'capital': 'Moroni'},
{'timezones': ['Europe/Zagreb'], 'code': 'HR', 'continent': 'Europe', 'name': 'Croatia', 'capital': 'Zagreb'},
{'timezones': ['Asia/Dili'], 'code': 'TL', 'continent': 'Asia', 'name': 'East Timor', 'capital': 'Dili'},
{'timezones': ['America/El_Salvador'], 'code': 'SV', 'continent': 'North America', 'name': 'El Salvador', 'capital': 'San Salvador'},
{'timezones': ['Africa/Malabo'], 'code': 'GQ', 'continent': 'Africa', 'name': 'Equatorial Guinea', 'capital': 'Malabo'},
{'timezones': ['America/Grenada'], 'code': 'GD', 'continent': 'North America', 'name': 'Grenada', 'capital': "St. George's"},
{'timezones': ['Asia/Almaty', 'Asia/Qyzylorda', 'Asia/Aqtobe', 'Asia/Aqtau', 'Asia/Oral'], 'code': 'KZ', 'continent': 'Asia', 'name': 'Kazakhstan', 'capital': 'Astana'},
{'timezones': ['Asia/Vientiane'], 'code': 'LA', 'continent': 'Asia', 'name': 'Laos', 'capital': 'Vientiane'},
{'timezones': ['Pacific/Truk', 'Pacific/Ponape', 'Pacific/Kosrae'], 'code': 'FM', 'continent': 'Oceania', 'name': 'Federated States of Micronesia', 'capital': 'Palikir'},
{'timezones': ['Europe/Chisinau'], 'code': 'MD', 'continent': 'Europe', 'name': 'Moldova', 'capital': 'Chi\xc5\x9fin\xc4\x83u'},
{'timezones': ['Europe/Monaco'], 'code': 'MC', 'continent': 'Europe', 'name': 'Monaco', 'capital': 'Monaco'},
{'timezones': ['Europe/Podgorica'], 'code': 'ME', 'continent': 'Europe', 'name': 'Montenegro', 'capital': 'Podgorica'},
{'timezones': ['Africa/Casablanca'], 'code': 'MA', 'continent': 'Africa', 'name': 'Morocco', 'capital': 'Rabat'},
{'timezones': ['America/St_Kitts'], 'code': 'KN', 'continent': 'North America', 'name': 'Saint Kitts and Nevis', 'capital': 'Basseterre'},
{'timezones': ['America/St_Lucia'], 'code': 'LC', 'continent': 'North America', 'name': 'Saint Lucia', 'capital': 'Castries'},
{'timezones': ['America/St_Vincent'], 'code': 'VC', 'continent': 'North America', 'name': 'Saint Vincent and the Grenadines', 'capital': 'Kingstown'},
{'timezones': ['Pacific/Apia'], 'code': 'WS', 'continent': 'Oceania', 'name': 'Samoa', 'capital': 'Apia'},
{'timezones': ['Europe/Belgrade'], 'code': 'RS', 'continent': 'Europe', 'name': 'Serbia', 'capital': 'Belgrade'},
{'timezones': ['Africa/Johannesburg'], 'code': 'ZA', 'continent': 'Africa', 'name': 'South Africa', 'capital': 'Pretoria'},
{'timezones': ['Europe/Madrid', 'Africa/Ceuta', 'Atlantic/Canary'], 'code': 'ES', 'continent': 'Europe', 'name': 'Spain', 'capital': 'Madrid'},
{'timezones': ['Asia/Colombo'], 'code': 'LK', 'continent': 'Asia', 'name': 'Sri Lanka', 'capital': 'Sri Jayewardenepura Kotte'},
{'timezones': ['Africa/Mbabane'], 'code': 'SZ', 'continent': 'Africa', 'name': 'Swaziland', 'capital': 'Mbabane'},
{'timezones': ['Europe/Zurich'], 'code': 'CH', 'continent': 'Europe', 'name': 'Switzerland', 'capital': 'Bern'},
{'timezones': ['Asia/Dubai'], 'code': 'AE', 'continent': 'Asia', 'name': 'United Arab Emirates', 'capital': 'Abu Dhabi'},
{'timezones': ['Europe/London'], 'code': 'GB', 'continent': 'Europe', 'name': 'United Kingdom', 'capital': 'London'},
]

In [154]:
country_capital_city_tz = dict()
[country_capital_city_tz.update({name['name']:name['continent'].split()[-1]+"/"+"_".join(name['capital'].split(" ")).replace(",","")}) for name in countries_info]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [155]:
country_capital_city_tz['France']

'Europe/Paris'

In [156]:
country_capital_city_tz['United States']

'America/Washington_D.C.'

In [157]:
import json
with open('country_capital_city_tz.json', 'w') as file:
     file.write(json.dumps(country_capital_city_tz))

In [158]:
!head country_capital_city_tz.json


{"Canada": "America/Ottawa", "East Timor": "Asia/Dili", "Turkmenistan": "Asia/Ashgabat", "Lithuania": "Europe/Vilnius", "Cambodia": "Asia/Phnom_Penh", "Saint Kitts and Nevis": "America/Basseterre", "Ethiopia": "Africa/Addis_Ababa", "The Gambia": "Africa/Banjul", "Sri Lanka": "Asia/Sri_Jayewardenepura_Kotte", "Swaziland": "Africa/Mbabane", "Argentina": "America/Buenos_Aires", "Bolivia": "America/Sucre", "Cameroon": "Africa/Yaound\u00e9", "Burkina Faso": "Africa/Ouagadougou", "Ghana": "Africa/Accra", "Saudi Arabia": "Asia/Riyadh", "Laos": "Asia/Vientiane", "Japan": "Asia/Tokyo", "Republic of Ireland": "Europe/Dublin", "Slovenia": "Europe/Ljubljana", "Guatemala": "America/Guatemala_City", "Bosnia and Herzegovina": "Europe/Sarajevo", "Kuwait": "Asia/Kuwait_City", "Germany": "Europe/Berlin", "Dominica": "America/Roseau", "Liberia": "Africa/Monrovia", "Maldives": "Asia/Mal\u00e9", "Pakistan": "Asia/Islamabad", "Oman": "Asia/Muscat", "Tanzania": "Africa/Dodoma", "Seychelles": "Africa/Victoria

### Add TimeZone information for each session )

### SWITCHING TO Map Reduce

In [159]:
import ast
str_ = "'\\xe5\\x9d\\x82\\xe6\\x9c\\xac\\xe9\\xbe\\x8d\\xe4\\xb8\\x80'"
str_[1:-1]

'\\xe5\\x9d\\x82\\xe6\\x9c\\xac\\xe9\\xbe\\x8d\\xe4\\xb8\\x80'

In [160]:
%%writefile MRTranslate.py

import os

import mrjob
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob import step
from mrjob.protocol import RawProtocol

from collections import defaultdict, Counter

import re
import sys
import json

import datetime
from datetime import datetime
from dateutil import parser
from dateutil import tz
import pytz

import ast

#import goslate 
#import translate
from googletrans import Translator

import requests


class MRTranslate(MRJob):

    def __init__(self, *args, **kwargs):
        super(MRTranslate, self).__init__(*args, **kwargs)
        self.min_line_length = 12
        self.translator   = Translator()
        self.day_of_week  = {0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"}
        self.time_of_day  = {"Night":[0,1,2,3,4], "Dawn":[5,6],"Morning":[7,8,9],"LateMorning":[10,11], "Lunch":[12,13],"Afternoon":[14,15,16],"LateAfternoon":[17,18],"Evening":[19,20,21],"LateEvening":[22,23]}

    
    def configure_options(self):
        super(MRTranslate, self).configure_options()
        self.add_file_option('--tzFile', default='/user/w210/data/country_capital_city_tz.json', dest='tzFile')
        self.add_passthrough_option('--num-mappers',  default=10, type="int", dest="num_mappers",  help="Number of mappers")
        self.add_passthrough_option('--num-reducers', default=10, type="int", dest="num_reducers", help="Number of reducers")
        self.add_passthru_arg('--local_dir',    default='/user/hduser',  dest="local_dir",  help="Working directory for user")
        self.add_passthru_arg('--tmp_dir',      default='/user/hduser',  dest="tmp_dir",    help="Temp directory for user")                                    
        
    #def configure_args(self):
    #    super(MRTranslate, self).configure_args()
    #    self.add_file_arg('--tzFile', default='/user/w210/data/country_capital_city_tz.json', dest='tzFile')
    #    self.add_passthru_arg('--num_mappers',  default=10,  dest="num_mappers",  help="Number of mappers")
    #    self.add_passthru_arg('--num_reducers', default=10,  dest="num_reducers", help="Number of reducers")
    #    self.add_passthru_arg('--local_dir',    default='/user/hduser',  dest="local_dir",  help="Working directory for user")
    #    self.add_passthru_arg('--tmp_dir',      default='/user/hduser',  dest="tmp_dir",    help="Temp directory for user")
        
    def translate(self, text):
        try:
            result = self.translator.translate(text)
        except:
            result = "Null"
        return result
    
    def utc_to_local(self, utc_timestamp, local_tz):
        try:
            local_dt = utc_timestamp.replace(tzinfo=pytz.utc).astimezone(local_tz)
            #sys.stderr.write("LOCAL_DT:"+repr(local_dt)+"\n")
            result = local_tz.normalize(local_dt)
            #sys.stderr.write("RESULT ====> "+repr(result)+"\n")
        except:
            result = None
        return result
    
    def get_time_of_day(self, hour):
        for key in self.time_of_day.keys():
            if hour in self.time_of_day[key]:
                return key

    def mapper_translate(self, _, line_):
        
        tzFile = str(self.options.tzFile) 
        
        tz_dict = dict()
        
        #sys.stderr.write("About to OPEN tzFILE"+"\n")
        
        with open(tzFile,'r') as f:
            
            #sys.stderr.write("OPENED tzFILE"+"\n")
            
            lines     = f.readlines()
            
            tz_dict   = ast.literal_eval(lines[0])
    
        if len(line_) > self.min_line_length:
            
            line      = line_.split("\t")
            user_id   = list(line)[0]
            value     = [v.replace('"','').strip() for v in line[1:]]
            utc_time  = value[0][:-1]
            traid     = value[1]
            try:
                #artname   = self.translator.translate(value[2]).text
                artname   = value[2]
            except:
                artname   = value[2]
            something = value[3]
            try:
                #tracname  = self.translator.translate(value[4]).text
                tracname  = value[4]
            except:
                tracname  = value[4]
            session_window = value[5]
            gender    = value[6]
            age       = value[7]
            country   = value[8]
            registered= value[9] 
            reg_date  = value[10]
                
                
            try:
                local_tz  = pytz.timezone(tz_dict[country])
            except:
                local_tz  = None
            try:
                utc    = datetime.strptime(utc_time, '%Y-%m-%dT%H:%M:%S')
            except: 
                utc    = None
                
            try:
                event_time = self.utc_to_local(utc, local_tz)
            except: 
                event_time = None
                    
            try:
                time_diff  = event_time.utcoffset()
                local_time = event_time + time_diff
            except:
                local_time = "Null"
                
            try:
                day_of_week = self.day_of_week[local_time.weekday()]
            except:
                day_of_week = "Null"
                    
            try:
                time_of_day = self.get_time_of_day(local_time.hour)
            except:
                time_of_day = "Null"
                    
            #sys.stderr.write(" utc_time: " + repr(str(utc_time)) + " traid:  "    + repr(str(traid))     + " artname: "     + repr(str(artname)) +"\n")
            #sys.stderr.write(" artname:  " + repr(str(artname))  + " something: " + repr(str(something)) +  "\n")
            #sys.stderr.write(" tracname: " + repr(str(tracname)) + " session_window: " + repr(str(session_window)) + " gender: " + repr(str(gender)) + "\n")
            #sys.stderr.write(" age: "      + repr(str(age))      + " country:"   + repr(str(country))    + " registered: "  + repr(str(registered))  + "\n")
            #sys.stderr.write(" reg_date: " + repr(str(reg_date)) + " local_tz: " + repr(str(local_tz))   + " local_time: "  + str(local_time) + " time_of_day: " + str(time_of_day) + " day_of_week: " +  str(day_of_week)+"\n")
            #sys.stderr.write("============================================================================================================================")
            #sys.stderr.write("============================================================================================================================")
                    
            value_ = []
                
            for x in [utc_time, traid, artname, something, tracname, session_window, gender, age, country, registered, reg_date, local_tz, local_time, time_of_day, day_of_week]:
                try:
                    value_.append(repr(x))
                except:
                    value_.append("Null")
                        
            #sys.stderr.write("\t".join([repr(x) for x in value_]))
                
            try:
                yield user_id, value_

            except:
                yield user_id, None
        
        




    def steps(self):
        
        JOBCONF = {
            'mapreduce.job.maps': self.options.num_mappers,
            'mapreduce.job.reduces': self.options.num_reducers,
            'mapreduce.cluster.local.dir': '/user/hduser/tmpdir',
            'mapreduce.cluster.temp.dir':  '/user/hduser/tmpdir',
            'mapreduce.partition.keypartitioner.options': '-k1',
            'mapreduce.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapreduce.text.key.comparator.options': '-k1',
            'mapreduce.map.memory.mb':40960,
            'mapreduce.reduce.memory.mb':40960,
            'mapreduce.map.java.opts':'-Xmx30720m',
            'mapreduce.reduce.java.opts':'-Xmx61440m'
        }
        JOBCONF1 = {
            'mapreduce.job.maps': self.options.num_mappers,
            'mapred.reduce.tasks': self.options.num_reducers,
            'mapreduce.partition.keypartitioner.options': '-k1',
            'mapreduce.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapreduce.text.key.comparator.options': '-k1',
            'mapreduce.map.memory.mb':40960,
            'mapreduce.reduce.memory.mb':40960,
            'mapreduce.map.java.opts':'-Xmx30720m',
            'mapreduce.reduce.java.opts':'-Xmx61440m'
        }
        
        return [MRStep(mapper=self.mapper_translate,
                       jobconf=JOBCONF)]
       

if __name__ == '__main__':
    MRTranslate.run()


Overwriting MRTranslate.py


In [161]:
!head -n 5 data/Session_data_1k.csv

userid	timestamp	artid	artname_	traid	traname_	session_window	gender	age	country	registered	reg_date
user_000001	2009-05-04T23:08:57Z	f1b1cf71-bd35-4e99-8624-24a6e15f133a	Deep Dish		Fuck Me Im Famous (Pacha Ibiza)-09-28-2007	23	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:54:10Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Composition 0919 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:52:04Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc2 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:42:52Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Hibari (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13


In [162]:
!ls -alrth  data/Session_data_*

-rw-r--r-- 1 hduser hadoop 153K Sep 16 11:15 data/Session_data_1k.csv
-rw-r--r-- 1 hduser hadoop 1.6M Sep 16 11:15 data/Session_data_10k.csv
-rw-r--r-- 1 hduser hadoop  17M Sep 16 11:15 data/Session_data_100k.csv
-rw-r--r-- 1 hduser hadoop 1.4K Sep 16 12:45 data/Session_data_10.csv
-rw-r--r-- 1 hduser hadoop 164M Sep 16 20:17 data/Session_data_1M.csv


In [163]:
!export PATH=$PATH:/usr/local/hadoop/bin:/usr/local/hadoop/sbin

In [164]:
!export HADOOP_opts="-Dhadoop.tmp.dir=/path/"

In [165]:
TRANSLATE_PY ='/data_data/session_length/MRTranslate.py'

In [166]:
JAR_FILE = '/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.0.1.jar'

In [167]:
USER = !whoami
USER = 'w210'
OUTPUT_PATH_BASE = '/user/{USER}'.format(USER=USER)

In [168]:
OUTPUT_DIR = "{OUTPUT_PATH_BASE}/data/output" 

In [169]:
TEST_1k = "{OUTPUT_PATH_BASE}/data/Session_data_1k.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)
TEST_10k = "{OUTPUT_PATH_BASE}/data/Session_data_10k.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)
TEST_100k = "{OUTPUT_PATH_BASE}/data/Session_data_100k.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)
TEST_1M = "{OUTPUT_PATH_BASE}/data/Session_data_1M.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)
FULL_DATA = "{OUTPUT_PATH_BASE}/data/Session_data.csv".format(OUTPUT_PATH_BASE=OUTPUT_PATH_BASE)

In [170]:
!hdfs dfs -ls /user/w210/data/Session*

-rw-r--r--   1 hduser supergroup 3298819885 2018-09-18 16:53 /user/w210/data/Session_data.csv
-rw-r--r--   1 hduser supergroup   16905208 2018-09-18 16:53 /user/w210/data/Session_data_100k.csv
-rw-r--r--   1 hduser supergroup    1610078 2018-09-18 16:53 /user/w210/data/Session_data_10k.csv
-rw-r--r--   1 hduser supergroup  171621932 2018-09-18 16:53 /user/w210/data/Session_data_1M.csv
-rw-r--r--   1 hduser supergroup     156293 2018-09-18 16:53 /user/w210/data/Session_data_1k.csv


In [171]:
!echo {TEST_1k}

/user/w210/data/Session_data_1k.csv


In [172]:
!hdfs dfs -ls {TEST_1k}

-rw-r--r--   1 hduser supergroup     156293 2018-09-18 16:53 /user/w210/data/Session_data_1k.csv


In [173]:
!ls -alrth country_capital*

-rw-r--r-- 1 hduser hadoop 6.3K Sep 16 16:07 country_capital_city_tz
-rw-r--r-- 1 hduser hadoop 6.1K Sep 27 15:23 country_capital_city_tz.json


In [174]:
TZ_FILE='/user/w210/data/country_capital_city_tz.json'

In [175]:
{TZ_FILE}

{'/user/w210/data/country_capital_city_tz.json'}

In [176]:
#!hadoop fs -rm -r /user/w210/data/output2
#!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.0.1.jar \
#    -D mapreduce.job.maps=10 \
#    -D hadoop.tmp.dir=file:/data_data/session_length/hadoop/tmp \
#    -files MRTranslate.py\
#    -input hdfs:///user/w210/data/Session_data_1M.csv \
#    -output /user/w210/data/output2 \
#    -numReduceTasks 10 \
#    -cmdenv tzFile={TZ_FILE}

In [177]:
!rm /data_data/session_length/session_data_1M.tsv
#!rm -R /tmp/MRtranslate*
!python MRTranslate.py ./data/Session_data_1M.csv  --tzFile=/data_data/session_length/country_capital_city_tz.json --tmp_dir=/data_data/session_length/hadoop/tmp  > session_data_1M.tsv

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
No handlers could be found for logger "mrjob.launch"
Using configs in /home/hduser/.mrjob.conf
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
Running step 1 of 1...
Creating temp directory /data_data/session_length/hadoop/tmp/MRTranslate.hduser.20180927.152331.683796
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_a

configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrou

configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrou

configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
add_passthrough_option() is deprecated and will be removed in v0.7.0. Use add_passthru_arg() instead.
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
add_passthrou

In [178]:
!head session_data_1M.tsv

"user_000001"	["'2006-08-13T17:28:03'", "'69158f97-4c07-4c4e-baf8-4e4ab1ed666e'", "'Boards Of Canada'", "'a3c49027-eb58-4e16-8848-28a857f936cd'", "'Oscar See Through Red Eye'", "'17'", "'m'", "''", "'Japan'", "'Aug 13, 2006'", "'2006-08-13'", "<DstTzInfo 'Asia/Tokyo' LMT+9:19:00 STD>", "datetime.datetime(2006, 8, 14, 11, 28, 3, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>)", "'LateMorning'", "'Monday'"]
"user_000001"	["'2006-08-13T17:23:11'", "'69158f97-4c07-4c4e-baf8-4e4ab1ed666e'", "'Boards Of Canada'", "'90da05fc-9d90-4244-8ac3-6ddeb0cf1fb5'", "'Hey Saturday Sun'", "'17'", "'m'", "''", "'Japan'", "'Aug 13, 2006'", "'2006-08-13'", "<DstTzInfo 'Asia/Tokyo' LMT+9:19:00 STD>", "datetime.datetime(2006, 8, 14, 11, 23, 11, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>)", "'LateMorning'", "'Monday'"]
"user_000001"	["'2006-08-13T17:14:49'", "'69158f97-4c07-4c4e-baf8-4e4ab1ed666e'", "'Boards Of Canada'", "'f9dc952e-ded2-4c48-be73-bc17127aafa8'", "'Farewell Fire'", "'17'", "'m'", "''", "'

## IMPORT HOLIDAY DATA FOR EACH COUNTRY

#### List of International Holidays created in "Import National Holidays.ipynb"

##### Format of Holidays_DF

Date: for each country we included every day in the year

Country: Every country in the timeanddate.com database is included [same data used in ios/apple]

is_holiday: Flag set to 0 if Date == holiday, otherwise set to 1


In [179]:
Holidays_DF = pd.read_csv('/data_data/session_length/international_holidays.tsv', sep='\t').iloc[:,2:6].set_index(['country','date']).unstack()

In [180]:
Holidays_DF.head(10)

is_holiday                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     \
date                Apr-1-2008 Apr-1-2009 Apr-10-2009 Apr-11-2008 Apr-11-2009 Apr-12-2008 Apr-12-2009 Apr-13-2008 Apr-13-2009 Apr-14-2008 Apr-14-2009 Apr-15-2008 Apr-15-2009 Apr-16-2008 Apr-16-2009 Apr-17-2008 Apr-17-2009 Apr-18-2008 Apr-18-2009 Apr-19-2008 Apr-19-2009 Apr-2-2008 Apr-2-2009 Apr-20-2008 Apr-20-2009 Apr-21-2008 Apr-21-2009 Apr-22-2008 Apr-22-2009 Apr-23-2008 Apr-23-2009 Apr-24-2008 Apr-24-2009 Apr-25-2008 Apr-25-2009 Apr-26-2008 Apr-26-2009 Apr-27-2008 Apr-27-2009 Apr-28-2008 Apr-28-2009 Apr-29-2008 Apr-29-2009 Apr-3-2008 Apr-3-2009 Apr-30-2008 Apr-30-2009 Apr-4-2008 Apr-4-2009 Apr-5-2008 Apr-5-2009 Apr-6-2008 Apr-6-2009 Apr-7-2008 Apr-7-2009 Apr-8-2008 Apr-8-2009 Apr-9-2008 Apr-9-2009 Aug-1-2008 Aug-1-2009 Aug-10-2008 Aug-10-2009 Aug-11-2008 Aug-12-2008 Aug-12-2009 Aug-13-2008 Aug-13-2009 Aug-14-2008 Aug-14-2009 Aug-15-2008 Aug-15-2009 Aug-16-2008 Aug-16-2009 Aug-17-2008 Aug-17-2009 Aug-18-2008 Aug-19-2008 Aug-19-2009 Aug-2-2008 Aug-2-2009 Aug-20-2008 Aug-20-2009   
country                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
algeria                      0          0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0          0          0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0          0          0           0           0          0          0          0          0          0          0          0          0          0          0          0          0          0          0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0          0          0           0           0   
american samoa               0  

#### You can query Holidays_DF as follows (if not using JOIN)

In [181]:
country = 'algeria'

In [182]:
date = 'Dec-25-2008'

In [183]:
query_prefix = 'is_holiday'

In [184]:
ctry_holidays_dict = dict(Holidays_DF.loc[country])

In [185]:
ctry_holidays_dict['is_holiday',date]

0

In [186]:
int_holidays_dict = dict(Holidays_DF)

In [187]:
int_holidays_dict['is_holiday', date][country]

0

#### NOTE that some countries have alternate names

In [188]:
exceptions = {"czech republic":"czech","bosnia and herzegovina":"bosnia","alsace":"france","eswatini":"swaziland",
              "united states":"us","united kingdom":"uk","wales":"uk","scotland":"uk", "northern ireland":"uk",
              "congo democratic republic":"dr-congo","equatorial guinea":"guineaecuatorial", "congo":"republic-of-the-congo",
              "east timor":"timor-leste", "trinidad and tobago":"trinidad","the bahamas":"bahamas", "comoros":"comores",
              "us virgin islands":"united-states-virgin-islands","cabo verde":"cape-verde","st. barts":"saint-barthelemy",
              "russian federation":"russia", "cote d'ivoire":"ivory coast", "korea, democratic people's republic of":"north korea",
              "congo, the democratic republic of the":"dr-congo"}

In [189]:
countries_with_holidays_dict = int_holidays_dict['is_holiday', date].keys()

In [190]:
def get_holiday_status(country, date):
    
    try:
        date = datetime.strptime(date, '%Y-%m-%d').strftime('%b-%d-%Y')
    except:
        try:
            date = date.strftime('%b-%d-%Y')
        except:
            pass
    try:
    
        country = country.lower()

        if country in countries_with_holidays_dict:

            return int_holidays_dict['is_holiday', date][country]

        else:

            if country.lower().startswith("the"):

                country = country.lower().replace("the ","")

                try:
                    return int_holidays_dict['is_holiday', date][country]
                except:
                    return get_holiday_status(country, date)

            elif country in exceptions:

                country = exceptions[country]

                try:

                    return int_holidays_dict['is_holiday', date][country]

                except:

                    return get_holiday_status(country, date)

            elif "-" in country.lower():

                country = " ".join(country.lower().split("-"))

                try:

                    return int_holidays_dict['is_holiday', date][country]

                except:

                    return get_holiday_status(country, date)
            else:
                #Need to fix this default option
                #For now, we are returning '0' IF country not in Holidays_DICT
                return 0
    except:
        
        return 0

In [191]:
user_sessions_DF = pd.read_csv('/data_data/session_length/usersessions-jayashree.csv')

In [192]:
user_sessions_DF.head()

,userid,gender,age,country,startdate,day_of_week,timeofday,sessionid,session_start,session_length
0,user_000001,m,NaN,Japan,2006-08-13,sunday,noon,1,2006-08-13 13:59:20,230
1,user_000001,m,NaN,Japan,2006-08-15,tuesday,morning,1,2006-08-15 12:27:51,170
2,user_000001,m,NaN,Japan,2006-08-16,wednesday,morning,1,2006-08-16 10:27:16,43
3,user_000001,m,NaN,Japan,2006-08-16,wednesday,noon,2,2006-08-16 13:43:02,86
4,user_000001,m,NaN,Japan,2006-08-17,thursday,noon,1,2006-08-17 13:33:27,115


In [193]:
countries_in_sessions = list([ctry for ctry in sessionsDF.select('country').distinct().toPandas()['country'] if ctry])

In [194]:
countries_in_sessions

[u'British Indian Ocean Territory',
 u'Congo, the Democratic Republic of the',
 u'Sweden',
 u'Singapore',
 u'Turkey',
 u'Germany',
 u'Northern Mariana Islands',
 u'France',
 u'Greece',
 u'Algeria',
 u'Slovakia',
 u'Argentina',
 u'Belgium',
 u'Finland',
 u'Nicaragua',
 u'Peru',
 u'United States',
 u'India',
 u'China',
 u'Belarus',
 u'Malta',
 u'Chile',
 u'Croatia',
 u'Italy',
 u'Netherlands Antilles',
 u'Lithuania',
 u'Norway',
 u'Spain',
 u'Russian Federation',
 u'Ireland',
 u'Thailand',
 u'Morocco',
 u'Venezuela',
 u'Iceland',
 u'Israel',
 u"Cote D'Ivoire",
 u'Antarctica',
 u'Mexico',
 u'Estonia',
 u'Zimbabwe',
 u'Armenia',
 u'Tunisia',
 u'Trinidad and Tobago',
 u'Switzerland',
 u'Latvia',
 u'Canada',
 u'Macedonia',
 u'Czech Republic',
 u'Brazil',
 u'Slovenia',
 u'Japan',
 u'New Zealand',
 u'United States Minor Outlying Islands',
 u'Bosnia and Herzegovina',
 u'Poland',
 u'Portugal',
 u'Australia',
 u'Romania',
 u'Bulgaria',
 u'Austria',
 u'Serbia',
 u"Korea, Democratic People's Republ

In [195]:
get_holiday_status('united states',date)

1

In [196]:
int_holidays_dict['is_holiday', date]

country
algeria                             0
american samoa                      1
andorra                             1
angola                              1
anguilla                            1
antigua and barbuda                 1
argentina                           1
armenia                             0
australia                           1
austria                             1
azerbaijan                          0
bangladesh                          1
barbados                            1
belarus                             1
belgium                             1
belize                              1
benin                               1
bolivia                             1
bosnia                              1
botswana                            1
brazil                              1
british virgin islands              1
brunei                              1
bulgaria                            1
cambodia                            0
cameroon                            1
cana

##### Check that we have holiday data for each country

In [213]:
get_holiday_status_udf = udf(lambda country, date:  int(get_holiday_status(country,date)), IntegerType())

In [214]:
session_with_holiday_DF = sessionsDF.withColumn('is_holiday',get_holiday_status_udf(col('country'),col('session_date')))

In [215]:
session_with_holiday_DF.printSchema()

root
 |-- userid: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- artid: string (nullable = true)
 |-- artname_: string (nullable = true)
 |-- traid: string (nullable = true)
 |-- traname_: string (nullable = true)
 |-- session_date: date (nullable = true)
 |-- session_window: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- registered: string (nullable = true)
 |-- reg_date: date (nullable = true)
 |-- is_holiday: integer (nullable = true)



In [217]:
session_with_holiday_DF.show()

+-----------+--------------------+--------------------+------------------+--------------------+--------------------+------------+--------------+------+---+-------------+-----------+----------+----------+
|     userid|           timestamp|               artid|          artname_|               traid|            traname_|session_date|session_window|gender|age|      country| registered|  reg_date|is_holiday|
+-----------+--------------------+--------------------+------------------+--------------------+--------------------+------------+--------------+------+---+-------------+-----------+----------+----------+
|user_000066|2009-03-27T17:18:26Z|                null|   The Amen Corner|                null|Our Love (Is In T...|  2009-03-27|            17|     f| 20|United States|May 9, 2006|2006-05-09|         0|
|user_000066|2009-03-08T04:06:21Z|2ceb4e66-4eaa-4db...|Natalia Lafourcade|a8864539-e4cc-4fc...|          En El 2000|  2009-03-08|             4|     f| 20|United States|May 9, 2006|200

In [219]:
session_with_holiday_DF.sample(0.00001).show(100, False)

+-----------+--------------------+------------------------------------+---------------------------------+------------------------------------+------------------------------------------------------+------------+--------------+------+----+--------------+------------+----------+----------+
|userid     |timestamp           |artid                               |artname_                         |traid                               |traname_                                              |session_date|session_window|gender|age |country       |registered  |reg_date  |is_holiday|
+-----------+--------------------+------------------------------------+---------------------------------+------------------------------------+------------------------------------------------------+------------+--------------+------+----+--------------+------------+----------+----------+
|user_000577|2007-04-17T20:01:10Z|f181961b-20f7-459e-89de-920ef03c7ed0|The Strokes                      |4468200b-a01a-41a8-9a98-22150e0

### Save as file (via toPandas())

In [228]:
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
session_with_holiday_DF.toPandas().to_csv("session_with_holiday_all.csv", sep='\t', encoding='utf-8',index=False)
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [229]:
!head session_with_holiday_all.csv

userid	timestamp	artid	artname_	traid	traname_	session_date	session_window	gender	age	country	registered	reg_date	is_holiday
user_000066	2009-03-27T17:18:26Z		The Amen Corner		Our Love (Is In The Pocket)	2009-03-27	17	f	20.0	United States	May 9, 2006	2006-05-09	0
user_000066	2009-03-08T04:06:21Z	2ceb4e66-4eaa-4dba-ad3a-30df3b742557	Natalia Lafourcade	a8864539-e4cc-4fc7-93d5-59c3c6f19a64	En El 2000	2009-03-08	4	f	20.0	United States	May 9, 2006	2006-05-09	0
user_000066	2009-03-08T03:57:15Z	b539e453-c4fe-47e3-8a07-8517eac74429	宇多田ヒカル	7de5cd2c-cb9e-40e0-bdf7-768c757b3a35	Time Will Tell	2009-03-08	3	f	20.0	United States	May 9, 2006	2006-05-09	0
user_000066	2009-03-08T03:52:42Z	86443348-fd26-48cf-bd13-7ea7fa27505c	Crystal Kay	997cf64c-b1cf-4909-9c7d-752b47116141	あなたのそばで	2009-03-08	3	f	20.0	United States	May 9, 2006	2006-05-09	0
user_000066	2009-03-08T03:38:22Z	22a1161f-966c-4c38-aba0-c99cb59806de	Jyongri	46aacfaf-4e18-4a72-80d1-7d0a14a48e2d	Possession	2009-03-08	3	f	20.0	United States	May 9,